# AG4 - Actividad Guiada 4 - Algoritmos Genéticos

Nombre: Carlos Javier Bravo Intriago

Link:   https://colab.research.google.com/drive/1Yd5cj-AmDNnWgQpIo7UXIurUMpJbt1ZA?usp=sharing

Github: https://github.com/carlosbravo1408/03MIAR-Algoritmos-de-Optimizacion-2025/tree/main/AG4


# Carga de librerias

In [1]:
#!pip install requests
#!pip install tabulate>=0.9 networkx>=3.0
#!pip install tsplib95 --no-deps
#!pip install deprecated

# Importe de librerias


In [2]:
import heapq
import gzip
import os
import random
import shutil
import urllib.request
from heapq import heapify
from typing import Union, Tuple, List, LiteralString

import tsplib95

## Métodos Auxiliares y algunas definiciones necesarias


In [3]:
NumericType = Union[int, float]
NodeType = int
EdgeType = Tuple[NodeType, NodeType]
SolutionType = List[NodeType]
TSProblemType = tsplib95.models.Problem


def download_tsp_file(url: str, filename: str) -> None:
    # Descarga como archivo temporal
    local_path, headers = urllib.request.urlretrieve(url, filename + ".temp")
    mime_type = headers.get_content_type()
    # Descomprimir si es formato gzip
    if "gzip" in mime_type or local_path.endswith(".gz"):
        with gzip.open(local_path, 'rb') as f_in:
            with open(filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(local_path)
    # Caso contrario renombra el archivo temporal
    else:
        os.replace(local_path, filename)

# Carga de datos del problema

In [4]:
#DATOS DEL PROBLEMA
# Matriz Adyacencia swiss42 problem (Staedte Schweiz/Fricker)
file = "swiss42.tsp"
download_tsp_file(
    "https://raw.githubusercontent.com/mastqe/tsplib/refs/heads/master/swiss42.tsp",
    file)
tsp_problem: tsplib95.models.Problem = tsplib95.load(file)

# Lo siguiente NO es una buena práctica de programacion y hay gente que
# considera sucia, pero permite optimizar algo más el código. Se inyecta la
# lista de nodos en el objeto tsp_problem, con ello se evita crear una nueva
# lista cada vez que se invoca a `get_nodes()`
tsp_problem.cached_nodes = list(tsp_problem.get_nodes())

#Nodos
nodes = tsp_problem.cached_nodes

# Funciones básicas
> Funciones de la Actividad Guiada 3

In [5]:
def crear_solucion(nodos: List[NodeType]) -> SolutionType:
    """
    Se genera una solución aleatoria con comienzo en el nodo 0
    :param nodos: Lista de nodos
    :return: Lista de nodos en posiciones aleatorias sin repetición
    """
    solucion = [nodos[0]]
    for _ in nodos[1:]:
        solucion = solucion + [
            random.choice(list(set(nodos) - {nodos[0]} - set(solucion)))]
    return solucion

def distancia(a: int, b: int, problem: TSProblemType) -> NumericType:
    """
    Devuelve la distancia entre dos nodos
    :param a: Nodo a
    :param b: Nodo b
    :param problem: Instancia de tsplib95.models.Problem
    :return:
    """
    return problem.get_weight(a, b)

def distancia_total(solucion: SolutionType, problem: TSProblemType) -> NumericType:
    """
    Devuelve la distancia total de una trayectoria/solución
    :param solucion: Lista de Nodos
    :param problem: Instancia de tsplib95.models.Problem
    :return:
    """
    distancia_total = 0
    n = len(solucion)
    for i in range(len(solucion)):
        distancia_total += distancia(solucion[i], solucion[(i + 1)%n], problem)
    return distancia_total

def validar_solucion(solucion: SolutionType, problem: TSProblemType) -> bool:
    return set(solucion) == set(problem.cached_nodes)


distancia_total(crear_solucion(nodes), tsp_problem)

5053

# Funciones Auxiliares

In [9]:
def generar_poblacion(problem:TSProblemType, N: int) -> List[SolutionType]:
    """
    Genera una poblacion inicial de soluciones de tamaño N.
    Se propone generar un 85% de poblacion aleatoria, y el 15% restante una
    poblacion voraz, con una predisposición a tener mejor gen con respecto a
    sus contrapartes generadas aleatoriamente
    :param problem:
    :param N:
    :return:
    """
    nodos = problem.cached_nodes
    poblacion = []
    nodos_vistos = set()
    semilla = int(N * 0.15)
    poblacion.extend(generar_poblacion_voraz(problem, semilla))
    while len(poblacion) < N:
        # Evitar duplicados, se debe procurar que en la generacion de
        # poblacion no haya individuos genéticamente iguales (comparación
        # rápida y amortizado lineal es con Tuplas y Set)
        individuo = tuple(random.sample(nodos, len(nodos)))
        if individuo not in nodos_vistos:
            nodos_vistos.add(individuo)
            poblacion.append(list(individuo))
    return poblacion

def generar_poblacion_voraz(problem: TSProblemType, N: int) -> List[SolutionType]:
    nodos = problem.cached_nodes
    nodos_iniciales = random.sample(nodos, N)
    poblacion = []
    nodos_vistos = set()
    for nodo_inicial in nodos_iniciales:
        individuo = [nodo_inicial]
        no_visitados = set(nodos)
        no_visitados.remove(nodo_inicial)
        nodo_actual = nodo_inicial
        while no_visitados:
            siguiente_nodo = min(
                no_visitados, key=lambda candidato: distancia(nodo_actual, candidato, problem)
            )
            individuo.append(siguiente_nodo)
            no_visitados.remove(siguiente_nodo)
            nodo_actual = siguiente_nodo

        individuo = tuple(individuo)
        if individuo not in nodos_vistos:
            nodos_vistos.add(individuo)
            poblacion.append(list(individuo))
    return poblacion

poblacion_prueba = generar_poblacion(tsp_problem, 50)
assert validar_solucion(poblacion_prueba[0], tsp_problem)
_ = [print(p) for p in poblacion_prueba[:5]]

[2, 27, 3, 4, 6, 1, 0, 7, 37, 15, 14, 16, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 29, 30, 28, 32, 34, 20, 33, 31, 17, 36, 35, 38, 22, 39, 21, 40, 24]
[3, 2, 27, 28, 29, 30, 32, 0, 1, 6, 4, 26, 5, 19, 13, 18, 12, 11, 25, 10, 8, 9, 23, 41, 21, 39, 22, 38, 34, 20, 33, 31, 17, 37, 15, 14, 16, 7, 36, 35, 40, 24]
[8, 9, 23, 41, 25, 10, 12, 11, 18, 26, 5, 6, 1, 0, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 4, 7, 36, 35, 38, 22, 39, 21, 40, 24]
[25, 10, 8, 9, 23, 41, 11, 12, 18, 26, 5, 6, 1, 0, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 4, 7, 36, 35, 38, 22, 39, 21, 40, 24]
[11, 12, 18, 26, 5, 6, 1, 0, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 4, 7, 36, 35, 38, 22, 39, 21, 40, 24, 9, 23, 41, 25, 10, 8]


In [29]:
def Evaluar_Poblacion(
        poblacion: List[SolutionType],
        problem: TSProblemType
) -> Tuple[SolutionType, NumericType]:
    """
    Evalúa la población y devuelve el mejor individuo.\n
    Ingenuamente retorna el individuo con menor coste.
    :param poblacion:
    :param problem:
    :return:
    """
    sol = min(poblacion, key=lambda s: distancia_total(s, problem))
    dist = distancia_total(sol, problem)
    return sol, dist

Evaluar_Poblacion(generar_poblacion(tsp_problem, 50), tsp_problem)

([13,
  19,
  5,
  26,
  18,
  12,
  11,
  25,
  10,
  8,
  9,
  23,
  41,
  29,
  30,
  28,
  2,
  27,
  3,
  4,
  6,
  1,
  0,
  7,
  37,
  15,
  14,
  16,
  17,
  31,
  35,
  36,
  20,
  34,
  33,
  32,
  38,
  22,
  39,
  21,
  40,
  24],
 1564)

In [ ]:
def Cruzar(
        poblacion: List[SolutionType],
        mutacion: float,
        problem: TSProblemType
) -> List[SolutionType]:
    """
    Función de cruce. Recibe una poblacion(lista de soluciones) y devuelve la
    población ampliada con los hijos.\n
    Todos los individuos de la población son seleccionados para el cruce(si
    la población es par)\n
    Podría aplicarse un proceso previo de selección para elegir los
    individuos que se desea cruzar.
    :param poblacion:
    :param mutacion:
    :param problem:
    :return:
    """


In [ ]:
def Descendencia(
        padres: Tuple[SolutionType, SolutionType],
        problem: TSProblemType,
        mutacion: float,
        estrategia_cruce: LiteralString["1-point", "2-point"] = "2-point"
) -> Tuple[SolutionType, SolutionType]:
    """
    Función para generar hijos a partir de 2 padres:
    Se elige el método de 1-punto de corte pero es posible usar otros
    n-puntos, uniforme, dependiendo del problema
    :param padres:
    :param problem:
    :param mutacion:
    :param estrategia_cruce:
    :return:
    """
    padre_a = padres[0]
    padre_b = padres[1]

    divisor = random.randint(1, len(padres) - 1)

    # division
    hijo_a = padre_a[:divisor] + padre_b[divisor:]
    hijo_b = padre_b[:divisor] + padre_a[divisor:]

    # factibilizar
    hijo_a = Factibilizar(hijo_a, problem)
    hijo_b = Factibilizar(hijo_b, problem)

    # mutar
    hijo_a = Mutar(hijo_a, mutacion)
    hijo_b = Mutar(hijo_b, mutacion)

    return hijo_a, hijo_b



In [14]:
def Factibilizar(solucion: SolutionType, problem: TSProblemType) -> SolutionType:
    """
    Para el operador de cruce 1-punto los hijos generados no son soluciones
    (algunos nodos se repiten y otros no están)\n
    Método que verifica y corrige aquellos hijos generados en el cruce.
    :param solucion:
    :param problem:
    :return:
    """
    n = len(solucion)
    _solucion = list(solucion)
    all_nodes = set(range(n))
    presentes = set(_solucion)
    faltantes = list(all_nodes - presentes)
    random.shuffle(faltantes)
    auxiliar = set()
    for i in range(n):
        nodo_actual = _solucion[i]
        if nodo_actual in auxiliar:
            _solucion[i] = faltantes.pop()
        else:
            auxiliar.add(nodo_actual)
    validar_solucion(_solucion, problem)
    return _solucion


[1, 2, 3, 4, 5, 8, 6, 7, 0]

In [ ]:
def Mutar(solucion: SolutionType, mutacion: float):
    """
    Función de mutación. Se eligen dos nodos y se intercambia. Se podrían
    añadir otros operadores\n
    Se hace mutaciones mutación % de las veces
    :param solucion:
    :param mutacion:
    :return:
    """
    _solution = list(solucion)
    for mut in range(1):  # aqui gestionar la mutacion iterativamente
        i, j = sorted(random.sample(range(1, len(solucion)), 2))
        _solution[i], _solution[j] = _solution[j], _solution[i]
    return _solution

In [ ]:
def Seleccionar(
        problem: TSProblemType,
        poblacion: List[SolutionType],
        N: int,
        elitismo: float
):
    """
     Función de selección de la población. Recibe como parametro una
     poblacion y devuelve una poblacion a la que se ha eliminado individuos
     poco aptos(fitness alto) y para mantener una poblacion estable de N
     individuos\n
     Se tiene en cuenta el porcentaje elitismo pasado como parametro
     Para los individuos que no son de la elite podríamos usar una selección
     de ruleta(proporcional a su fitness)
    :param problem:
    :param poblacion:
    :param N:
    :param elitismo:
    :return:
    """
    sorted()

# Proceso Principal

In [ ]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(
        problem: TSProblemType,
        N: int = 100,
        mutacion: float = .15,
        elitismo: float = .1,
        generaciones: int = 100
):
    """

    :param problem: datos del problema
    :param N: Tamaño de la población
    :param mutacion: probabilidad de una mutación
    :param elitismo: porción de la mejor poblacion a mantener
    :param generaciones: nº de generaciones a generar para finalizar
    :return:
    """

    #Genera la poblacion inicial
    Nodos = problem.cached_nodes
    poblacion = generar_poblacion(Nodos, N)

    #Inicializamos valores para la mejor solucion
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    #Condicion de parada
    parar = False
    n = 0
    #Inciamos el ciclo de generaciones
    while (parar == False):

        #Cruce de la poblacion(incluye mutación)
        poblacion = Cruzar(poblacion, mutacion, problem)

        #Seleccionamos la población
        poblacion = Seleccionar(problem, poblacion, N, elitismo)

        #Evaluamos la nueva población
        (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(
            poblacion, problem
        )

        print(
            "Generación #", n, "\nLa mejor solución es:", mejor_solucion,
            "\ncon distancia ", mejor_distancia, "\n"
        )

        #Numero de generaciones. Criterio de parada
        if n == generaciones:
            parar = True
        n += 1

    return mejor_solucion


sol = algoritmo_genetico(problem=tsp_problem, N=500, mutacion=.3, elitismo=.40,
                         generaciones=250)